In [3]:
import torch

In [11]:
B, T, C = 2, 8, 16 # batch size, sequence length, embedding dimensionality (n_embd)


k = torch.randn(size=(B, T, C, C)).transpose(1, 2) # (B, nh, T, hs)
q = torch.randn(size=(B, T, C, C)).transpose(1, 2) # (B, nh, T, hs)
v = torch.randn(size=(B, T, C, C)).transpose(1, 2) # (B, nh, T, hs)


In [12]:





# causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)

    # efficient attention using Flash Attention CUDA kernels
y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True)
# y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

# # output projection
# y = self.resid_dropout(self.c_proj(y))


In [10]:
y.size()

torch.Size([2, 16, 8, 16])

In [ ]:


        attn_mask = None if is_causal else attn_mask.unsqueeze(1).unsqueeze(1).expand(batch_size, self.num_heads, seq_len, seq_len)
        
        if self.flash_attention:
            y = F.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, 
                is_causal=is_causal, dropout_p=self.dropout if self.training else 0.)
        else:
            y = self.scaled_dot_product_attention(q, k, v, seq_len, attn_mask=attn_mask, is_causal=is_causal)
            
        y = y.transpose(1, 2).contiguous().view(batch_size, seq_len, embedding_dim)        
        y = self.out(y)
        
        return y
    
    def scaled_dot_product_attention(self, q, k, v, seq_len, attn_mask, is_causal) -> torch.Tensor:
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        if is_causal:
            att = att.masked_fill(self.mask_causal[:, :, :seq_len, :seq_len] == 0, float('-inf'))
        else:
            att = att.masked_fill(attn_mask.int() == 0, float('-inf'))
        att_probs = F.softmax(att, dim=-1)
        y = self.attn_dropout(att_probs) @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        return y
    